Running a local nominatim server

docker run -it -v /var/tmp/ldh/nominatim:/data  -e PBF_PATH=/data/texas-latest.osm.pbf  -p 8080:8080   --name nominatimjyp1 mediagis/nominatim:5.1 

http://localhost:8080/search?q=14+Terrace+Pl,+San+Antonio,+TX&format=json

In [ ]:
!pip3 install geopy

In [ ]:
import pickle
import pandas as pd

In [ ]:
!date

In [ ]:
headers = [
    "Taxpayer_Number", "Taxpayer_Name", "Taxpayer_Address", "Taxpayer_City", "Taxpayer_State",
    "Taxpayer_Zip", "Taxpayer_County", "Taxpayer_Phone", "Location_Number", "Location_Name",
    "Location_Address", "Location_City", "Location_State", "Location_Zip", "Location_County",
    "Location_Phone", "Unit_Capacity", "Responsibility_Begin_Date", "Responsibility_End_Date",
    "Obligation_End_Date", "Filer_Type", "Total_Room_Receipts", "Taxable_Receipts"
]

In [ ]:
df = pd.read_csv(r'./HOT2505.CSV', low_memory=False, on_bad_lines='skip', encoding="latin-1", names=headers, header=0)

In [ ]:
!date

In [ ]:
from geopy.geocoders import Nominatim
from geopy.adapters import AioHTTPAdapter
import pandas as pd
import time


# Point to your local server
geolocator = Nominatim(
    user_agent="tx_hotels_map",
    domain="localhost:8080",
    scheme="http"
)

def get_coordinates(row):
    full_address = f"{row['Location_Address']}, {row['Location_City']}, TX {row['Location_Zip']}"
    try:
        location = geolocator.geocode(full_address, timeout=3)
#        time.sleep(1)  # Respect rate limits even locally
        if location:
            return pd.Series([location.latitude, location.longitude])
    except Exception as e:
        print(f"Error with address '{full_address}': {e}")
        return pd.Series([None, None])

df[["Latitude", "Longitude"]] = df.apply(get_coordinates, axis=1)


In [ ]:
!date

In [ ]:
df.info()

In [ ]:
with open(r'fullState2025.pkl','wb') as ofile:
    pickle.dump(df, ofile)

In [ ]:
with open(r'fullState2024.pkl','rb') as ifile:
    df = pickle.load(ifile)

In [ ]:
!date

In [ ]:
df.info()

In [ ]:
!pip3 install folium

In [ ]:
!date

In [ ]:
import folium

# Create the base map centered on San Antonio
hotel_map = folium.Map(location=[29.5187, -98.6047], zoom_start=12)

# Normalize receipts to set radius scale
min_receipt = df["Total_Room_Receipts"].min()
max_receipt = df["Total_Room_Receipts"].max()
range_receipt = max_receipt - min_receipt

# Add scaled circle markers
for _, row in df.dropna(subset=["Latitude", "Longitude"]).iterrows():
    # Scale radius between 4 and 20
    scaled_radius = 4 + 16 * ((row["Total_Room_Receipts"] - min_receipt) / range_receipt)

    # Color by receipt amount (simple logic: low = light, high = dark blue)
    if row["Total_Room_Receipts"] == 0:
        color = "gray"
    elif row["Total_Room_Receipts"] < 100000:
        color = "lightblue"
    elif row["Total_Room_Receipts"] < 300000:
        color = "blue"
    else:
        color = "darkblue"

    popup_text = f"""
    <b>{row['Location_Name']}</b><br>
    Receipts: ${row['Total_Room_Receipts']:,.2f}<br>
    Units: {row['Unit_Capacity']}
    """

    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=scaled_radius,
        color=color,
        fill=True,
        fill_color=color,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(hotel_map)

# Save the map
hotel_map.save("Texas_May24_receipts_scaled_map_3.html")


In [ ]:
!date

In [ ]:
!ls
!pwd